# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 02:56:57] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38284, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=72541

[2025-08-03 02:56:57] Using default HuggingFace chat template with detected content format: string


[2025-08-03 02:57:06] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 02:57:06] Init torch distributed begin.


[2025-08-03 02:57:07] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 02:57:08] Load weight begin. avail mem=62.17 GB


...

[2025-08-03 02:57:12] Init torch distributed begin.
[2025-08-03 02:57:12] Init torch distributed ends. mem usage=0.00 GB
[2025-08-03 02:57:12] Load weight begin. avail mem=38.65 GB
[2025-08-03 02:57:12] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.16s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.16s/it]

[2025-08-03 02:57:13] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=37.72 GB, mem usage=0.93 GB.
[2025-08-03 02:57:13] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-03 02:57:13] Memory pool end. avail mem=37.40 GB


[2025-08-03 02:57:14] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=37.63 GB
[2025-08-03 02:57:14] INFO:     Started server process [1441599]
[2025-08-03 02:57:14] INFO:     Waiting for application startup.
[2025-08-03 02:57:14] INFO:     Application startup complete.
[2025-08-03 02:57:14] INFO:     Uvicorn running on http://127.0.0.1:38284 (Press CTRL+C to quit)


[2025-08-03 02:57:14] INFO:     127.0.0.1:47090 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 02:57:15] INFO:     127.0.0.1:47096 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 02:57:15] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 02:58:20] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-03 02:58:50] INFO:     127.0.0.1:47108 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 02:58:50] The server is fired up and ready to roll!


[2025-08-03 02:58:53] INFO:     127.0.0.1:35870 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 02:59:00] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=38755, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=5885789

[2025-08-03 02:59:01] Using default HuggingFace chat template with detected content format: string


[2025-08-03 02:59:09] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 02:59:09] Init torch distributed begin.


[2025-08-03 02:59:20] Init torch distributed ends. mem usage=-17.19 GB


[2025-08-03 02:59:21] Load weight begin. avail mem=78.58 GB


...

Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.17s/it]

[2025-08-03 02:59:27] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-08-03 02:59:27] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-03 02:59:27] Memory pool end. avail mem=53.92 GB


[2025-08-03 02:59:27] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.10 GB
[2025-08-03 02:59:27] INFO:     Started server process [1449697]
[2025-08-03 02:59:27] INFO:     Waiting for application startup.
[2025-08-03 02:59:27] INFO:     Application startup complete.
[2025-08-03 02:59:27] INFO:     Uvicorn running on http://127.0.0.1:38755 (Press CTRL+C to quit)


[2025-08-03 02:59:28] INFO:     127.0.0.1:41948 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 02:59:28] INFO:     127.0.0.1:41962 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 02:59:28] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 02:59:32] INFO:     127.0.0.1:41974 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 02:59:32] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 02:59:33] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 02:59:34] INFO:     127.0.0.1:41978 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 02:59:42] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32184, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-03 02:59:43] Using default HuggingFace chat template with detected content format: string


[2025-08-03 02:59:50] Casting torch.bfloat16 to torch.float16.


[2025-08-03 02:59:51] Casting torch.bfloat16 to torch.float16.
[2025-08-03 02:59:51] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 02:59:51] Init torch distributed begin.


[2025-08-03 02:59:53] Init torch distributed ends. mem usage=-17.05 GB


...

[2025-08-03 03:00:13] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=42.85 GB


[2025-08-03 03:00:14] INFO:     Started server process [1452527]
[2025-08-03 03:00:14] INFO:     Waiting for application startup.
[2025-08-03 03:00:14] INFO:     Application startup complete.
[2025-08-03 03:00:14] INFO:     Uvicorn running on http://127.0.0.1:32184 (Press CTRL+C to quit)


[2025-08-03 03:00:15] INFO:     127.0.0.1:52220 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 03:00:15] INFO:     127.0.0.1:52236 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 03:00:15] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 03:00:18] INFO:     127.0.0.1:52244 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 03:00:18] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 03:00:20] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 03:00:20] INFO:     127.0.0.1:36746 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 03:00:29] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=37384, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-03 03:00:29] Using default HuggingFace chat template with detected content format: string


[2025-08-03 03:00:37] Casting torch.bfloat16 to torch.float16.


[2025-08-03 03:00:38] Casting torch.bfloat16 to torch.float16.
[2025-08-03 03:00:38] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 03:00:38] Init torch distributed begin.


[2025-08-03 03:00:39] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-03 03:00:58] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=58.56 GB


[2025-08-03 03:00:58] INFO:     Started server process [1454716]
[2025-08-03 03:00:58] INFO:     Waiting for application startup.
[2025-08-03 03:00:58] INFO:     Application startup complete.
[2025-08-03 03:00:58] INFO:     Uvicorn running on http://127.0.0.1:37384 (Press CTRL+C to quit)


[2025-08-03 03:00:59] INFO:     127.0.0.1:48086 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-03 03:00:59] INFO:     127.0.0.1:48102 - "GET /get_model_info HTTP/1.1" 200 OK


[2025-08-03 03:00:59] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 03:01:03] INFO:     127.0.0.1:48116 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 03:01:03] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-03 03:01:04] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 03:01:05] INFO:     127.0.0.1:48120 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-03 03:01:12] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=31943, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=753894132, constrained_jso

[2025-08-03 03:01:13] Using default HuggingFace chat template with detected content format: string


[2025-08-03 03:01:21] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-03 03:01:21] Init torch distributed begin.


[2025-08-03 03:01:21] Init torch distributed ends. mem usage=0.00 GB


[2025-08-03 03:01:22] Load weight begin. avail mem=78.58 GB


...

[2025-08-03 03:01:27] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=59.84 GB


[2025-08-03 03:01:28] INFO:     Started server process [1455754]
[2025-08-03 03:01:28] INFO:     Waiting for application startup.
[2025-08-03 03:01:28] INFO:     Application startup complete.
[2025-08-03 03:01:28] INFO:     Uvicorn running on http://0.0.0.0:31943 (Press CTRL+C to quit)


[2025-08-03 03:01:29] INFO:     127.0.0.1:54900 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-03 03:01:29] INFO:     127.0.0.1:54912 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-03 03:01:29] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 03:01:30] INFO:     127.0.0.1:54926 - "POST /generate HTTP/1.1" 200 OK
[2025-08-03 03:01:30] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-03 03:01:34] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-03 03:01:34] Decode batch. #running-req: 1, #token: 89, token usage: 0.00, accept len: 1.71, cuda graph: False, gen throughput (token/s): 10.21, #queue-req: 0, 


[2025-08-03 03:01:35] Decode batch. #running-req: 1, #token: 157, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 106.08, #queue-req: 0, 


[2025-08-03 03:01:36] Decode batch. #running-req: 1, #token: 225, token usage: 0.01, accept len: 1.70, cuda graph: False, gen throughput (token/s): 105.55, #queue-req: 0, 


[2025-08-03 03:01:36] Decode batch. #running-req: 1, #token: 294, token usage: 0.01, accept len: 1.73, cuda graph: False, gen throughput (token/s): 108.69, #queue-req: 0, 


[2025-08-03 03:01:37] Decode batch. #running-req: 1, #token: 358, token usage: 0.02, accept len: 1.60, cuda graph: False, gen throughput (token/s): 99.08, #queue-req: 0, 


[2025-08-03 03:01:37] Decode batch. #running-req: 1, #token: 429, token usage: 0.02, accept len: 1.77, cuda graph: False, gen throughput (token/s): 113.40, #queue-req: 0, 


[2025-08-03 03:01:38] INFO:     127.0.0.1:54938 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).